<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/ttmf_vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 5.6MB 5.4MB/s 
     |████████████████████████████████| 184kB 43.8MB/s 
     |████████████████████████████████| 3.2MB 53.6MB/s 
ERROR: requests 2.23.0 has requirement chardet<4,>=3.0.2, but you'll have chardet 4.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
!pip -q install xlsxwriter

     |████████████████████████████████| 153kB 5.4MB/s 


In [6]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [7]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [8]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [9]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 18


In [10]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

Permission denied: https://drive.google.com/uc?id=1IRcDf9ACIKKSs0a0aVHF7il5S6CCtZ9f
Maybe you need to change permission over 'Anyone with the link'?


In [11]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [12]:
landline_masterlist = landline.make_masterlist()

In [13]:
# ttmf daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 08 Quarantine Facility Data - Daily Report.csv'
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

In [14]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])

In [15]:
df_cap.keys()

Index(['updateddate', 'addeddate', 'reportdate', 'other_adm', 'doctor_adm',
       'nurse_adm', 'qother', 'qdoctor', 'qnurse', 'isolbed_o', 'isolbed_v',
       'beds_ward_o', 'beds_ward_v', 'facilityname', 'region', 'province',
       'city_mun', 'bgy', 'region_psgc', 'province_psgc', 'city_mun_psgc',
       'bgy_psgc', 'cap_reportdate'],
      dtype='object')

In [16]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [17]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [18]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.isolbed_v > 0) 
  df_vacant = df_cap[filter][['cap_reportdate', 'facilityname', 'city_mun', 'province',
                              'isolbed_v', 'beds_ward_v', 
                              'isolbed_o', 'beds_ward_o']]
  df_vacant.sort_values(by='isolbed_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [19]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,facilityname,city_mun,province,isolbed_v,beds_ward_v,isolbed_o,beds_ward_o
0,1276243,2021-04-15,HOPE 6 COMMUNITY CARING FACILITY,QUEZON CITY,"NCR, SECOND DISTRICT (NOT A PROVINCE)",151,0,105,0
1,1990760,2021-04-15,PAMANTASAN NG LUNGSOD NG MARIKINA TEMPORARY TR...,CITY OF MARIKINA,"NCR, SECOND DISTRICT (NOT A PROVINCE)",88,0,0,0
2,1882614,2021-04-15,AIRFORCE GENERAL HOSPITAL ISOLATION FACILITY,PASAY CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",85,0,15,145
3,364821,2021-04-15,NCRPO SPECIAL CARE FACILITY,TAGUIG CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",72,0,49,0
4,1951759,2021-04-15,WE HEAL AS ONE CENTER-MUNTINLUPA CITY,CITY OF MUNTINLUPA,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",65,0,0,0
5,1848198,2021-04-15,MOA ISOLATION FACILITY,PASAY CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",64,0,96,0
6,994232,2021-04-15,MATAAS NA PAARALANG NEPTALI A. GONZALES,CITY OF MANDALUYONG,"NCR, SECOND DISTRICT (NOT A PROVINCE)",31,0,8,0
7,2386715,2021-04-15,TONDO HIGH SCHOOL QUARANTINE FACILITY,TONDO I / II,"NCR, CITY OF MANILA, FIRST DISTRICT (NOT A PRO...",20,4,20,0
8,1755683,2021-04-15,FABELLA HIGH SCHOOL QUARANTINE FACILITY,CITY OF MANDALUYONG,"NCR, SECOND DISTRICT (NOT A PROVINCE)",20,0,0,0
9,564516,2021-04-15,MARCELO GREEN ISOLATION FACILITY,CITY OF PARAÑAQUE,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",19,4,17,0


In [20]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,facilityname,city_mun,province,isolbed_v,beds_ward_v,isolbed_o,beds_ward_o
0,728877,2021-04-16,MACARIO B ASISTIO HIGH SCHOOL ISOLATION FACILITY,CALOOCAN CITY,"NCR, THIRD DISTRICT (NOT A PROVINCE)",107,0,0,0
1,1882615,2021-04-16,AIRFORCE GENERAL HOSPITAL ISOLATION FACILITY,PASAY CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",85,0,15,145
2,1848199,2021-04-16,MOA ISOLATION FACILITY,PASAY CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",74,0,86,0
3,364822,2021-04-16,NCRPO SPECIAL CARE FACILITY,TAGUIG CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",73,0,48,0
4,1951760,2021-04-16,WE HEAL AS ONE CENTER-MUNTINLUPA CITY,CITY OF MUNTINLUPA,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",60,0,0,0
5,1550186,2021-04-16,BAGONG SIBOL QUARANTINE FACILITY,CITY OF MARIKINA,"NCR, SECOND DISTRICT (NOT A PROVINCE)",46,0,98,0
6,994233,2021-04-16,MATAAS NA PAARALANG NEPTALI A. GONZALES,CITY OF MANDALUYONG,"NCR, SECOND DISTRICT (NOT A PROVINCE)",42,0,8,0
7,1755684,2021-04-16,FABELLA HIGH SCHOOL QUARANTINE FACILITY,CITY OF MANDALUYONG,"NCR, SECOND DISTRICT (NOT A PROVINCE)",26,0,0,0
8,2386716,2021-04-16,TONDO HIGH SCHOOL QUARANTINE FACILITY,TONDO I / II,"NCR, CITY OF MANILA, FIRST DISTRICT (NOT A PRO...",21,4,19,0
9,1459995,2021-04-16,HOPE 2 COMMUNITY CARING FACILITY,QUEZON CITY,"NCR, SECOND DISTRICT (NOT A PROVINCE)",19,0,0,0


In [21]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,facilityname,city_mun,province,isolbed_v,beds_ward_v,isolbed_o,beds_ward_o
0,1882616,2021-04-17,AIRFORCE GENERAL HOSPITAL ISOLATION FACILITY,PASAY CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",85,0,15,145
1,364823,2021-04-17,NCRPO SPECIAL CARE FACILITY,TAGUIG CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",83,0,38,0
2,1848200,2021-04-17,MOA ISOLATION FACILITY,PASAY CITY,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",75,0,85,0
3,1951761,2021-04-17,WE HEAL AS ONE CENTER-MUNTINLUPA CITY,CITY OF MUNTINLUPA,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",67,0,0,0
4,994234,2021-04-17,MATAAS NA PAARALANG NEPTALI A. GONZALES,CITY OF MANDALUYONG,"NCR, SECOND DISTRICT (NOT A PROVINCE)",40,0,8,0
5,2386717,2021-04-17,TONDO HIGH SCHOOL QUARANTINE FACILITY,TONDO I / II,"NCR, CITY OF MANILA, FIRST DISTRICT (NOT A PRO...",30,5,10,0
6,1755685,2021-04-17,FABELLA HIGH SCHOOL QUARANTINE FACILITY,CITY OF MANDALUYONG,"NCR, SECOND DISTRICT (NOT A PROVINCE)",22,0,0,0
7,728878,2021-04-17,MACARIO B ASISTIO HIGH SCHOOL ISOLATION FACILITY,CALOOCAN CITY,"NCR, THIRD DISTRICT (NOT A PROVINCE)",20,0,0,0
8,1459996,2021-04-17,HOPE 2 COMMUNITY CARING FACILITY,QUEZON CITY,"NCR, SECOND DISTRICT (NOT A PROVINCE)",19,0,0,0
9,564518,2021-04-17,MARCELO GREEN ISOLATION FACILITY,CITY OF PARAÑAQUE,"NCR, FOURTH DISTRICT (NOT A PROVINCE)",12,4,24,0


In [22]:
def dump_daily(df_cap, days_ago):
  filter = (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) 
  df_vacant = df_cap[filter][['facilityname', 'region', 'province', 'city_mun', 
                              'isolbed_v', 'beds_ward_v', 
                              'isolbed_o', 'beds_ward_o', 
                              'addeddate', 'updateddate']]
  df_vacant.sort_values(by='isolbed_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True, drop=True)
  return df_vacant

In [23]:
#https://stackoverflow.com/questions/29463274/simulate-autofit-column-in-xslxwriter
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

In [24]:
writer = pd.ExcelWriter('ttmf_DOHDataDrop.xlsx', engine='xlsxwriter')
workbook  = writer.book
format_wrap = workbook.add_format({'text_wrap': True})

In [25]:
readme = 'README'
notes = []
notes.append('Data source: DOH Data Drop ' + data_date.strftime('%Y-%m-%d'))
notes.append('Data source link is in the README PDF @ ' + 'https://bit.ly/DataDropPH')
notes.append('Info here is dated -- call the ONE HOSPITAL COMMAND CENTER for current availability. Reach them through the ff. numbers: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('cap_reportdate = date of report for isolation rooms, beds in ward')
notes.append('This is not an official document. It has not been validated - there could be errors.')
notes.append('Refer to ncovtracker.doh.gov.ph for official announcements')
notes.append('Very important: Please contact the ONE HOSPITAL COMMAND CENTER if you are looking for a hospital room / healthcare referral.')
notes.append('OHCC contact: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('DOH COVID-19 toll-free hotline number: 1555')
notes.append('Contact your LGU - https://www.rappler.com/nation/list-covid-19-hotline-numbers-one-hospital-command-metro-manila')
notes.append('Source code at https://github.com/maytlim/doh_data')
df_readme = pd.DataFrame(notes)
df_readme.to_excel(writer, readme)

In [26]:
for days_ago in range(1, 4):
  reportdate = day_ago_from(days_ago, data_date)
  df_dumpdaily = dump_daily(df_cap, days_ago)
  df_dumpdaily.to_excel(writer, reportdate)
  worksheet = writer.sheets[reportdate]
  worksheet.set_row(0, None, cell_format=format_wrap)
  for i, width in enumerate(get_col_widths(df_dumpdaily)):
    worksheet.set_column(i, i, min(20, max(width + 4, 5)))
  (max_row, max_col) = df_dumpdaily.shape
  # Set the autofilter
  worksheet.autofilter(0, 1, max_row, max_col)

In [27]:
writer.close()